# PS 5 grading script
Notes: 
- You will want to have a Grading folder in your Google Drive with a subfolder of ps5
- Download the spreadsheets from canvas and put them in the Grading/ps5 folder in your Google Drive
- The solution file is in Kevin's dropbox folder (link is below)

https://colab.research.google.com/github/kpcrotty/mgmt648/blob/main/PS5_grader.ipynb

#### Mount Google Drive and list contents

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive/Grading/"

#### Packages and File Paths

In [ ]:
import os
import pandas as pd
import numpy as np
from scipy.stats import norm
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

######################################
#File paths
######################################
PATH  = '/content/drive/MyDrive/Grading'

# solution_file = PATH  + '/Week 5 Spreadsheet Assignment - SOLUTION.xlsx'
solution_file = 'https://www.dropbox.com/scl/fi/zjzsispbs9i4r82twwc7u/Week-5-Spreadsheet-Assignment-SOLUTION.xlsx?rlkey=e33gmtrg4fwxu1v2y47h6l7q1&st=pkvyjj6r&dl=1'
ps_path       = PATH + '/ps5'
output_file   = PATH + '/grades_ps5.xlsx'

#### Read in solutions

In [2]:
######################################
#Read in solutions
######################################
cells_partA = ['C' + str(i) for i in np.arange(14,20)] + ['C21']
cells_partB = ['C33','C35','C37','C39']
cells_partC = ['C44','C46'] 
cells_partD = ['C51','C53'] 
cells_partE = ['C' + str(i) for i in np.arange(61,67)]
cell_check = ['B66']
cells_to_pull1 = cells_partA + cells_partB + cells_partC + cells_partD + cells_partE + cell_check
cells_to_pull = cells_to_pull1
solution1 = pd.DataFrame(index=cells_to_pull, columns=['q1'])
for i in enumerate(cells_to_pull):  
    col_to_pull =str(cells_to_pull[i[0]][0])
    row_to_pull =int(cells_to_pull[i[0]][1:])-1       
    solution1.iloc[i[0],0]=pd.read_excel(solution_file,sheet_name='Portfolio Optimization', usecols=col_to_pull,skiprows=row_to_pull,nrows=1,header=None).iloc[0,0]

#### Read in student solutions 

- should be downloaded and unzipped from canvas and place in Grading/ps5 folder of google drive

In [ ]:
######################################
# Read in student submissions
# Downloaded and unzipped from canvas    
######################################    
#Pull file names for the particular folder path
file_list = os.listdir(ps_path)
os.chdir(ps_path)
zeros = np.zeros(len(file_list), dtype=[('q_1err','f8'), ('q_A_expl','str'),  ('q_A','f8'),('q_B','f8'),('q_C','f8'),('q_D','f8'),('q_E','f8')])
deductions = pd.DataFrame(zeros)    
deductions['feedback'] = ''

TOLERANCE = 0.10  #how far off the magnitudes can be for grading @risk output


for f in enumerate(file_list):
# for f in enumerate([file_list[i] for i in [6,10,16]]):
    print(f[1])


    ##### Question (All parts):
    cells_to_pull = cells_to_pull1
    ans1 = pd.DataFrame(index=cells_to_pull, columns=['q_1'])
    try:
        for i in enumerate(cells_to_pull):
            col_to_pull =str(cells_to_pull[i[0]][0])
            row_to_pull =int(cells_to_pull[i[0]][1:])-1        
            #try: 
            ans1.iloc[i[0],0]=pd.read_excel(f[1],sheet_name='Portfolio Optimization', usecols=col_to_pull,skiprows=row_to_pull,nrows=1,header=None).iloc[0,0]  
    except:
        print('Q_1 Read error')
        deductions.loc[f[0],'q_1err']=1  

    try:    
        if deductions.loc[f[0],'q_1err']==0:
            
            #Check for row insertion
            if ans1.loc['B66'][0]!=solution1.loc['B66'][0]:
                print('Q1 row problem')
                deductions.loc[f[0],'q_1err']=1   
            
            
            
            ##### Part A               
            w1 = ans1.loc['C14'][0]
            w2 = ans1.loc['C15'][0]
            #Tangency weights correct?;
            if not(round(ans1.loc['C14'][0],2)==round(solution1.loc['C14'][0],2)):
                    print('A: Wrong tangency weights')
                    deductions.loc[f[0],'feedback'] += '\n'+ 'A: Wrong tangency weights'
                    deductions.loc[f[0],'q_A'] = deductions.loc[f[0],'q_A']-0.5
    
            #Expected Return correct?
            if not(round(ans1.loc['C16'][0],2)==round(w1*0.08 + w2*0.05,2)):
                    print('A: E[r] wrong')
                    deductions.loc[f[0],'feedback'] += '\n'+'A: E[r] wrong'
                    deductions.loc[f[0],'q_A'] = deductions.loc[f[0],'q_A']-0.5
    
            #Standard deviation correct?
            if not(round(ans1.loc['C18'][0],2)==round(np.sqrt((w1**2)*0.35**2 + (w2**2)*0.15**2 + 2*w1*w2*0.3*0.35*0.15) ,2)):
                    print('A: SD wrong')
                    deductions.loc[f[0],'feedback'] += '\n'+'A: SD wrong'
                    deductions.loc[f[0],'q_A'] = deductions.loc[f[0],'q_A']-0.5
    
            #Sharpe ratio correct?
            if not(round(ans1.loc['C19'][0],2)==round( (ans1.loc['C16'][0]-0.01)/ans1.loc['C18'][0]  ,2)):
                    print('A: SR wrong')         
                    deductions.loc[f[0],'feedback'] += '\n'+'A: SR wrong'       
                    deductions.loc[f[0],'q_A'] = deductions.loc[f[0],'q_A']-0.5
            deductions.loc[f[0],'q_A_expl']=ans1.loc['C21'][0]            
            
            
            
            
            
            ##### Part B
            #Check Expected returns and SD of simulated market
            cell = 'C33'           
            if not(ans1.loc[cell][0]<(1+TOLERANCE)*(solution1.loc[cell][0]) and ans1.loc[cell][0]>(1-TOLERANCE)*(solution1.loc[cell][0])):
                    print('B: sim error of market return mean')
                    deductions.loc[f[0],'feedback'] += '\n'+'B: sim error of market return mean'       
                    deductions.loc[f[0],'q_B'] = deductions.loc[f[0],'q_B']-0.5
            cell = 'C35'           
            if not(ans1.loc[cell][0]<(1+TOLERANCE)*(solution1.loc[cell][0]) and ans1.loc[cell][0]>(1-TOLERANCE)*(solution1.loc[cell][0])):
                    print('B: sim error of market return sd')
                    deductions.loc[f[0],'feedback'] += '\n'+'B: sim error of market return sd'       
                    deductions.loc[f[0],'q_B'] = deductions.loc[f[0],'q_B']-0.5     
                    
            #Check Expected returns and SD of simulated part A portfolio        
            cell = 'C37'           
            if not(ans1.loc[cell][0]<(1+TOLERANCE)*(round(ans1.loc['C16'][0],2)) and ans1.loc[cell][0]>(1-TOLERANCE)*(round(ans1.loc['C16'][0],2))):
                    print('B: sim error of part A portfolio return mean')        
                    deductions.loc[f[0],'feedback'] += '\n'+'B: sim error of part A portfolio return mean'               
                    deductions.loc[f[0],'q_B'] = deductions.loc[f[0],'q_B']-0.5      
            cell = 'C39'           
            if not(ans1.loc[cell][0]<(1+TOLERANCE)*(round(ans1.loc['C18'][0],2)) and ans1.loc[cell][0]>(1-TOLERANCE)*(round(ans1.loc['C18'][0],2))):
                    print('B: sim error of part A portfolio return sd')
                    deductions.loc[f[0],'feedback'] += '\n'+'B: sim error of part A portfolio return sd'       
                    deductions.loc[f[0],'q_B'] = deductions.loc[f[0],'q_B']-0.5                       
             

            ##### Part C
            #Check fraction negative returns - market
            cell = 'C44'               
            correct = norm.cdf(0, loc=0.05, scale=0.15)
            if not(ans1.loc[cell][0]<(1+TOLERANCE)*correct and ans1.loc[cell][0]>(1-TOLERANCE)*correct):
                    print('C: market fraction negative error')     
                    deductions.loc[f[0],'feedback'] += '\n'+'C: market fraction negative error'                 
                    deductions.loc[f[0],'q_C'] = deductions.loc[f[0],'q_C']-0.5
            #Check fraction negative returns - part A portfolio
            cell = 'C46'               
            correct = norm.cdf(0, loc=ans1.loc['C16'][0], scale=ans1.loc['C18'][0])   # for partial credit
            if not(ans1.loc[cell][0]<(1+TOLERANCE)*correct and ans1.loc[cell][0]>(1-TOLERANCE)*correct):
                    print('C: part A portfolio fraction negative error')   
                    deductions.loc[f[0],'feedback'] += '\n'+'C: part A portfolio fraction negative error'                   
                    deductions.loc[f[0],'q_C'] = deductions.loc[f[0],'q_C']-0.5



            ##### Part D
            #Check fraction > 0.2 - market
            cell = 'C51'               
            correct = 1-norm.cdf(0.2, loc=0.05, scale=0.15)
            if not(ans1.loc[cell][0]<(1+TOLERANCE)*correct and ans1.loc[cell][0]>(1-TOLERANCE)*correct):
                    print('D: market fraction > 0.20 error')    
                    deductions.loc[f[0],'feedback'] += '\n'+'D: market fraction > 0.20 error'                                        
                    deductions.loc[f[0],'q_D'] = deductions.loc[f[0],'q_D']-0.5
            #Check fraction > 0.2 - part A portfolio
            cell = 'C53'               
            correct = 1-norm.cdf(0.2, loc=ans1.loc['C16'][0], scale=ans1.loc['C18'][0])   # for partial credit
            if not(ans1.loc[cell][0]<(1+TOLERANCE)*correct and ans1.loc[cell][0]>(1-TOLERANCE)*correct):
                    print('D: part A portfolio fraction > 0.20 error')   
                    deductions.loc[f[0],'feedback'] += '\n'+'D: part A portfolio fraction > 0.20 error'
                    deductions.loc[f[0],'q_D'] = deductions.loc[f[0],'q_D']-0.5
                    
                    
                    


            ##### Part E            
            w1 = ans1.loc['C61'][0]
            w2 = ans1.loc['C62'][0]
            w3 = ans1.loc['C63'][0]
            #Tangency weights correct?;
            if not(round(ans1.loc['C61'][0],2)==round(solution1.loc['C61'][0],2)):
                    print('E: Wrong tangency weights')   
                    deductions.loc[f[0],'feedback'] += '\n'+'E: Wrong tangency weights'             
                    deductions.loc[f[0],'q_E'] = deductions.loc[f[0],'q_E']-0.5
    
            #Expected Return correct?
            if not(round(ans1.loc['C64'][0],2)==round(w1*0.08 + w2*0.05 + w3*0.01,2)):
                    print('E: E[r] wrong')    
                    deductions.loc[f[0],'feedback'] += '\n'+'E: E[r] wrong'            
                    deductions.loc[f[0],'q_E'] = deductions.loc[f[0],'q_E']-0.5
    
            #Standard deviation correct?
            if not(round(ans1.loc['C65'][0],2)==round(np.sqrt((w1**2)*0.35**2 + (w2**2)*0.15**2 + 2*w1*w2*0.3*0.35*0.15) ,2)):
                    print('E: SD wrong') 
                    deductions.loc[f[0],'feedback'] += '\n'+'E: SD wrong'               
                    deductions.loc[f[0],'q_E'] = deductions.loc[f[0],'q_E']-0.5
    
            #Sharpe ratio correct?
            if not(round(ans1.loc['C66'][0],2)==round( (ans1.loc['C64'][0]-0.01)/ans1.loc['C65'][0]  ,2)):
                    print('E: SR wrong')  
                    deductions.loc[f[0],'feedback'] += '\n'+'E: SR wrong'  
                    deductions.loc[f[0],'q_E'] = deductions.loc[f[0],'q_E']-0.5       
            
            
        

            
    except:
        print('Q_1 Grade error')
        deductions.loc[f[0],'q_1err']=1   

#### Output grading file to excel and list results

In [ ]:
# Output to Excel  
deductions['sum']=10+deductions['q_A']+deductions['q_B']+deductions['q_C']+deductions['q_D']+deductions['q_E']
deductions['filename']=file_list
deductions.to_excel(output_file)    

In [ ]:
for i in deductions.index:    
    print('')
    print(deductions.loc[i,'filename'])
    x = deductions.loc[i,'sum']
    print(f"Total Score:\t{x}\n")
    x = deductions.loc[i,'q_A']
    print(f"QA Deduction:\t{x}")
    x = deductions.loc[i,'q_B']
    print(f"QB Deduction:\t{x}")
    x = deductions.loc[i,'q_C']
    print(f"QC Deduction:\t{x}")
    x = deductions.loc[i,'q_D']
    print(f"QD Deduction:\t{x}")
    x = deductions.loc[i,'q_E']
    print(f"QE Deduction:\t{x}\n")

    print(deductions.loc[i,'feedback'])
    print('')
    